This notebook sets up a simulation scene with a ycb object, records relevant camera data and saves the captured data in a serialized format to the drive.

In [1]:
from pathlib import Path

from alr_sim.core import Scene

from alr_sim_tools.ycb_utils import YCBLoader
from alr_sim_tools.scene_utils import (
    record_camera_data,
    reset_scene,
)

pybullet build time: May  2 2023 05:55:04


In [2]:
FACTORY_STRING = "mj_beta"
OBJECT_POS = (0.5, 0.0, 0.2)
CAM_POS = (0.5, 0.0, 1)
CAM_QUAT = [0.7071067811865476, 0.0, 0.0, -0.7071067811865475]
CAM_HEIGHT = 480
CAM_WIDTH = 640
ROBOT_POS = (0.0, 0.5, 0.2)
ROBOT_QUAT = (0, 1, 0, 0)
RENDER_MODE = Scene.RenderMode.HUMAN    
DATA_RECORDING_WAIT_TIME = 0.5
DATA_RECORDING_MOVE_TIME = 2
GRASP_PLANNER_SERVICE_ID = "contact_bench/contact_grasp_planner_service"

YCB_FOLDER = Path.home() / "Documents" / "SF-ObjectDataset" / "YCB"
YCB_OBJECT_ID = "011_banana"

In [3]:
ycb_loader = YCBLoader(ycb_base_folder=YCB_FOLDER, factory_string="mj_beta")

sim_obj = ycb_loader.get_ycb_object(
    pos=(0.5, 0.0, 0.2), quat=(0, 1, 0, 0), obj_id=YCB_OBJECT_ID, name=YCB_OBJECT_ID
)

In [4]:
camera_data, scene, agent = record_camera_data(
    factory_string=FACTORY_STRING,
    cam_pos=CAM_POS,
    cam_quat=CAM_QUAT,
    cam_height=CAM_HEIGHT,
    cam_width=CAM_WIDTH,
    robot_pos=ROBOT_POS,
    robot_quat=ROBOT_QUAT,
    object_list=[sim_obj],
    target_obj_name=YCB_OBJECT_ID,
    render_mode=RENDER_MODE,
    wait_time=DATA_RECORDING_WAIT_TIME,
    move_duration=DATA_RECORDING_MOVE_TIME,
)
reset_scene(FACTORY_STRING, scene, agent)